# Auto-sklearn Installation

In [ ]:
!apt-get install swig -y  # c or c++ -> python, ruby , etc ...
!pip install Cython numpy # 파이썬 코드를 C언어로 컴파일함
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip3 install
!sudo apt-get install build-essential swig

!pip install auto-sklearn==0.10.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   209  100   209    0     0   1632      0 --:--:-- --:--:-- --:--:--  1632
  Using cached scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
auto-sklearn 0.10.0 requires scikit-learn<0.23,>=0.22.0, but you have scikit-learn 0.24.2 which is inco

# Ready

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import time
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
import logging
from warnings import simplefilter

warnings.filterwarnings('ignore')
logging.captureWarnings(True)
simplefilter(action='ignore', category=FutureWarning)

# 데이터 처리

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/AI/clothes_size.csv')
data

,weight,age,height,size
0,62,28.0,172.72,XL
1,59,36.0,167.64,L
2,61,34.0,165.10,M
3,65,27.0,175.26,L
4,62,45.0,172.72,M
...,...,...,...,...
119729,63,42.0,175.26,M
119730,45,29.0,154.94,S
119731,61,31.0,172.72,M
119732,74,31.0,167.64,XL


In [ ]:
data_size = data.groupby('size').count()
data_size = data_size.index
data_size

Index(['L', 'M', 'S', 'XL', 'XXL', 'XXS', 'XXXL'], dtype='object', name='size')

In [ ]:
size_num = pd.DataFrame({'size' : data_size,
                        'num' : np.arange(1, len(data_size)+1)})
size_num

,size,num
0,L,1
1,M,2
2,S,3
3,XL,4
4,XXL,5
5,XXS,6
6,XXXL,7


In [ ]:
# size를 숫자로 표현
dataset = pd.merge(data, size_num)
dataset

,weight,age,height,size,num
0,62,28.0,172.72,XL,4
1,74,29.0,177.80,XL,4
2,64,32.0,160.02,XL,4
3,58,31.0,154.94,XL,4
4,68,36.0,162.56,XL,4
...,...,...,...,...,...
119729,68,24.0,149.86,XXL,5
119730,74,30.0,160.02,XXL,5
119731,69,32.0,160.02,XXL,5
119732,64,46.0,160.02,XXL,5


In [ ]:
# nan 제거
dataset.dropna(how='any', inplace=True)
dataset = dataset.drop(columns='age')
dataset

,weight,height,size,num
0,62,172.72,XL,4
1,74,177.80,XL,4
2,64,160.02,XL,4
3,58,154.94,XL,4
4,68,162.56,XL,4
...,...,...,...,...
119729,68,149.86,XXL,5
119730,74,160.02,XXL,5
119731,69,160.02,XXL,5
119732,64,160.02,XXL,5


# Autosklearn - 1800s

In [ ]:
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection

import autosklearn.classification

In [ ]:
X = dataset.drop(columns=['size', 'num'])
y = dataset['num']

In [ ]:
X.shape, y.shape

((119153, 2), (119153,))

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(89364, 2) (89364,)
(29789, 2) (29789,)


In [ ]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task = 1800,
    per_run_time_limit=30,
    tmp_folder='./cloth_size/autosklearn_classification_cloth_size_tmp1',
    output_folder='./cloth_size/autosklearn_classification_cloth_size_out1',
)
automl.fit(X_train, y_train)

[WARNING] [2021-07-20 05:44:15,379:EnsembleBuilder(1):84668e6a5505c8cc2ced45e20e3dbdcf] Memory Exception -- restart with less ensemble_nbest: 25
[WARNING] [2021-07-20 05:44:15,458:EnsembleBuilder(1):84668e6a5505c8cc2ced45e20e3dbdcf] Memory Exception -- restart with less ensemble_nbest: 12
[WARNING] [2021-07-20 05:44:15,534:EnsembleBuilder(1):84668e6a5505c8cc2ced45e20e3dbdcf] Memory Exception -- restart with less ensemble_nbest: 6
[WARNING] [2021-07-20 05:44:15,595:EnsembleBuilder(1):84668e6a5505c8cc2ced45e20e3dbdcf] Memory Exception -- restart with less ensemble_nbest: 3
[WARNING] [2021-07-20 05:44:15,674:EnsembleBuilder(1):84668e6a5505c8cc2ced45e20e3dbdcf] Memory Exception -- restart with less ensemble_nbest: 1
[CRITICAL] [2021-07-20 05:44:15,750:EnsembleBuilder(1):84668e6a5505c8cc2ced45e20e3dbdcf] Memory Exception -- Unable to further reduce the number of ensemble members -- please restart Auto-sklearn with a higher value for the argument 'ensemble_memory_limit' (current limit is 102

Exception ignored in: <function BackendContext.__del__ at 0x7f98dd103d40>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/autosklearn/util/backend.py", line 121, in __del__
    self.delete_directories(force=False)
  File "/usr/local/lib/python3.7/dist-packages/autosklearn/util/backend.py", line 129, in delete_directories
    % self.output_directory)
ValueError: Failed to delete output dir: ./tmp/autosklearn_classification_example_out1 because auto-sklearn did not create it. Please make sure that the specified output dir does not exist when instantiating auto-sklearn.
Exception ignored in: <function BackendContext.__del__ at 0x7f98dd103d40>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/autosklearn/util/backend.py", line 121, in __del__
    self.delete_directories(force=False)
  File "/usr/local/lib/python3.7/dist-packages/autosklearn/util/backend.py", line 129, in delete_directories
    % self.output_directory)
ValueE

AutoSklearnClassifier(dask_client=None,
                      delete_output_folder_after_terminate=True,
                      delete_tmp_folder_after_terminate=True,
                      disable_evaluator_output=False,
                      ensemble_memory_limit=1024, ensemble_nbest=50,
                      ensemble_size=50, exclude_estimators=None,
                      exclude_preprocessors=None, get_smac_object_callback=None,
                      include_estimators=None, include_preprocessors=None,
                      initi...
                      metadata_directory=None, metric=None,
                      ml_memory_limit=3072, n_jobs=None,
                      output_folder='./cloth_size/autosklearn_classification_cloth_size_out1',
                      per_run_time_limit=30, resampling_strategy='holdout',
                      resampling_strategy_arguments=None, seed=1,
                      smac_scenario_args=None, time_left_for_this_task=1800,
                      tmp_f

In [ ]:
print(automl.show_models())

[(1.000000, MyDummyClassifier(configuration=1, init_params=None, random_state=None)),
]


In [ ]:
predictions = automl.predict(X_test)
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))

Accuracy score: 0.14834334821578435


In [ ]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: 84668e6a5505c8cc2ced45e20e3dbdcf
  Metric: accuracy
  Best validation score: 0.502289
  Number of target algorithm runs: 117
  Number of successful target algorithm runs: 92
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 14
  Number of target algorithms that exceeded the memory limit: 5



# Autosklearn2 - 120s

In [ ]:
automl2 = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task = 120,
    per_run_time_limit=30,
    tmp_folder='./cloth_size/autosklearn_classification_cloth_size_tmp2',
    output_folder='./cloth_size/autosklearn_classification_cloth_size_out2',
)
automl2.fit(X_train, y_train)

AutoSklearnClassifier(dask_client=None,
                      delete_output_folder_after_terminate=True,
                      delete_tmp_folder_after_terminate=True,
                      disable_evaluator_output=False,
                      ensemble_memory_limit=1024, ensemble_nbest=50,
                      ensemble_size=50, exclude_estimators=None,
                      exclude_preprocessors=None, get_smac_object_callback=None,
                      include_estimators=None, include_preprocessors=None,
                      initi...
                      metadata_directory=None, metric=None,
                      ml_memory_limit=3072, n_jobs=None,
                      output_folder='./cloth_size/autosklearn_classification_cloth_size_out2',
                      per_run_time_limit=30, resampling_strategy='holdout',
                      resampling_strategy_arguments=None, seed=1,
                      smac_scenario_args=None, time_left_for_this_task=120,
                      tmp_fo

In [ ]:
print(automl2.show_models())

[(0.740000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'random_forest', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'standardize', 'feature_preprocessor:__choice__': 'no_preprocessing', 'classifier:random_forest:bootstrap': 'True', 'classifier:random_forest:criterion': 'gini', 'classifier:random_forest:max_depth': 'None', 'classifier:random_forest:max_features': 0.5, 'classifier:random_forest:max_leaf_nodes': 'None', 'classifier:random_forest:min_impurity_decrease': 0.0, 'classifier:random_forest:min_samples_leaf': 1, 'classifier:random_forest:min_samples_split': 2, 'classifier:random_forest:min_weight_fraction_leaf': 0.0, 'data_preprocessing:categorical_tran

In [ ]:
predictions = automl2.predict(X_test)
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))

Accuracy score: 0.5031051730504549


In [ ]:
print(automl2.sprint_statistics())

auto-sklearn results:
  Dataset name: 84668e6a5505c8cc2ced45e20e3dbdcf
  Metric: accuracy
  Best validation score: 0.501000
  Number of target algorithm runs: 6
  Number of successful target algorithm runs: 3
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0



# Autosklearn3 - 60s

In [ ]:
automl3 = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task = 60,
    per_run_time_limit=30,
    tmp_folder='./cloth_size/autosklearn_classification_cloth_size_tmp3',
    output_folder='./cloth_size/autosklearn_classification_cloth_size_out3',
)
automl3.fit(X_train, y_train)

[WARNING] [2021-07-20 06:20:56,810:AutoML(1):84668e6a5505c8cc2ced45e20e3dbdcf] Capping the per_run_time_limit to 29.0 to have time for a least 2 models in each process.
[WARNING] [2021-07-20 06:21:36,931:EnsembleBuilder(1):84668e6a5505c8cc2ced45e20e3dbdcf] Memory Exception -- restart with less ensemble_nbest: 25
[WARNING] [2021-07-20 06:21:38,031:EnsembleBuilder(1):84668e6a5505c8cc2ced45e20e3dbdcf] Memory Exception -- restart with less ensemble_nbest: 12
[WARNING] [2021-07-20 06:21:39,081:EnsembleBuilder(1):84668e6a5505c8cc2ced45e20e3dbdcf] Memory Exception -- restart with less ensemble_nbest: 6
[WARNING] [2021-07-20 06:21:40,121:EnsembleBuilder(1):84668e6a5505c8cc2ced45e20e3dbdcf] Memory Exception -- restart with less ensemble_nbest: 3
[WARNING] [2021-07-20 06:21:41,177:EnsembleBuilder(1):84668e6a5505c8cc2ced45e20e3dbdcf] Memory Exception -- restart with less ensemble_nbest: 1


AutoSklearnClassifier(dask_client=None,
                      delete_output_folder_after_terminate=True,
                      delete_tmp_folder_after_terminate=True,
                      disable_evaluator_output=False,
                      ensemble_memory_limit=1024, ensemble_nbest=50,
                      ensemble_size=50, exclude_estimators=None,
                      exclude_preprocessors=None, get_smac_object_callback=None,
                      include_estimators=None, include_preprocessors=None,
                      initi...
                      metadata_directory=None, metric=None,
                      ml_memory_limit=3072, n_jobs=None,
                      output_folder='./cloth_size/autosklearn_classification_cloth_size_out3',
                      per_run_time_limit=30, resampling_strategy='holdout',
                      resampling_strategy_arguments=None, seed=1,
                      smac_scenario_args=None, time_left_for_this_task=60,
                      tmp_fol

In [ ]:
print(automl3.show_models())

[(0.740000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'random_forest', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'standardize', 'feature_preprocessor:__choice__': 'no_preprocessing', 'classifier:random_forest:bootstrap': 'True', 'classifier:random_forest:criterion': 'gini', 'classifier:random_forest:max_depth': 'None', 'classifier:random_forest:max_features': 0.5, 'classifier:random_forest:max_leaf_nodes': 'None', 'classifier:random_forest:min_impurity_decrease': 0.0, 'classifier:random_forest:min_samples_leaf': 1, 'classifier:random_forest:min_samples_split': 2, 'classifier:random_forest:min_weight_fraction_leaf': 0.0, 'data_preprocessing:categorical_tran

In [ ]:
predictions = automl3.predict(X_test)
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))

Accuracy score: 0.5031051730504549


In [ ]:
print(automl3.sprint_statistics())

auto-sklearn results:
  Dataset name: 84668e6a5505c8cc2ced45e20e3dbdcf
  Metric: accuracy
  Best validation score: 0.501000
  Number of target algorithm runs: 3
  Number of successful target algorithm runs: 2
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

